Convert candidate TCRs from zheng 2021 CD4 single cell TCRs for getting predicted scores, in order to prepare for computing correlations

The TCRs should have v alleles belonging to one of the 142 human beta alleles in 

    ../data/for_encoders/combo_xcr.tsv
    
the original v part has a format different from what we used for DePTH model, we need to transform it

The amino acid sequences should not have strange characters

The amino acid sequences should not have length exceeding what we have encoded for

In [1]:
import pandas as pd
import numpy as np

from collections import defaultdict
from collections import Counter

In [2]:
data_dir = "../data/Zheng_2021/"

file_pos = "TCRs_CD4_pos_with_v_gene_weighted.csv"
file_neg = "TCRs_CD4_neg_with_v_gene_weighted.csv"

In [3]:
df_pos = pd.read_csv(data_dir + file_pos, header = 0)
df_neg = pd.read_csv(data_dir + file_neg, header = 0)

print(df_pos.shape)
print(df_neg.shape)

print(df_pos[:6])
print(df_neg[:6])

(6547, 1)
(20860, 1)
                        tcr
0  TRBV20-1,CSAHGGFANHHEQFF
1  TRBV20-1,CSAHGGFANHHEQFF
2  TRBV20-1,CSAHGGFANHHEQFF
3  TRBV20-1,CSAHGGFANHHEQFF
4  TRBV20-1,CSAHGGFANHHEQFF
5  TRBV20-1,CSAHGGFANHHEQFF
                        tcr
0  TRBV6-5,CASSYYPPPGQGEAFF
1  TRBV6-5,CASSYYPPPGQGEAFF
2  TRBV6-5,CASSYYPPPGQGEAFF
3  TRBV6-5,CASSYYPPPGQGEAFF
4  TRBV6-5,CASSYYPPPGQGEAFF
5  TRBV6-5,CASSYYPPPGQGEAFF


In [4]:
pos_v = [x.split(",")[0] for x in df_pos.tcr.tolist()]
pos_aa = [x.split(",")[1] for x in df_pos.tcr.tolist()]

neg_v = [x.split(",")[0] for x in df_neg.tcr.tolist()]
neg_aa = [x.split(",")[1] for x in df_neg.tcr.tolist()]

In [5]:
print(Counter([len(x) for x in pos_aa]))
print(Counter([len(x) for x in neg_aa]))

Counter({15: 1841, 14: 1787, 13: 1358, 16: 973, 12: 588})
Counter({15: 6102, 14: 5635, 13: 4069, 16: 3373, 12: 1681})


In [6]:
# add *01 to v gene to get the format as allowed in tcrdist -- why we want the format as in tcrdist? in combo_xcr.tsv?

all_vs = list(set(pos_v + neg_v))
len(all_vs)

50

In [7]:
all_vs.sort()
all_vs

['TRBV10-1',
 'TRBV10-2',
 'TRBV10-3',
 'TRBV11-1',
 'TRBV11-2',
 'TRBV11-3',
 'TRBV12-3',
 'TRBV12-4',
 'TRBV12-5',
 'TRBV13',
 'TRBV14',
 'TRBV15',
 'TRBV16',
 'TRBV18',
 'TRBV19',
 'TRBV2',
 'TRBV20-1',
 'TRBV21-1',
 'TRBV24-1',
 'TRBV25-1',
 'TRBV27',
 'TRBV28',
 'TRBV29-1',
 'TRBV3-1',
 'TRBV30',
 'TRBV4-1',
 'TRBV4-2',
 'TRBV4-3',
 'TRBV5-1',
 'TRBV5-4',
 'TRBV5-5',
 'TRBV5-6',
 'TRBV5-7',
 'TRBV5-8',
 'TRBV6-1',
 'TRBV6-3',
 'TRBV6-4',
 'TRBV6-5',
 'TRBV6-6',
 'TRBV6-7',
 'TRBV6-8',
 'TRBV6-9',
 'TRBV7-2',
 'TRBV7-3',
 'TRBV7-4',
 'TRBV7-6',
 'TRBV7-7',
 'TRBV7-8',
 'TRBV7-9',
 'TRBV9']

In [10]:
# load the combo_xcr.tsv table to translate the format to 
V_info = pd.read_csv("../data/for_encoders/combo_xcr.tsv", sep='\t')
V_sub_info = V_info.loc[(V_info.organism == 'human')
                        & (V_info.chain == 'B')
                        & (V_info.region == 'V')]
print(V_sub_info.shape)
V_sub_info[:6]

(142, 9)


,id,organism,chain,region,nucseq,frame,aligned_protseq,cdr_columns,cdrs
566,TRBV1*01,human,B,V,gatactggaattacccagacaccaaaatacctggtcacagcaatgg...,1,DTGITQTPKYLVTAMGSKRTMKREHLGH........DSMYWYRQKA...,27-38;56-65;81-86;104-109,GH........DS;YNC....KEF;P.DSSR;CTSSQ.
567,TRBV10-1*01,human,B,V,gatgctgaaatcacccagagcccaagacacaagatcacagagacag...,1,DAEITQSPRHKITETGRQVTLACHQTWNH.......NNMFWYRQDL...,27-38;56-65;81-86;104-109,WNH.......NN;SYG....VQD;S.NTED;CASSE.
568,TRBV10-1*02,human,B,V,gatgctgaaatcacccagagcccaagacacaagatcacagagacag...,1,DAEITQSPRHKITETGRQVTLACHQTWNH.......NNMFWYRQDL...,27-38;56-65;81-86;104-109,WNH.......NN;SYG....VHD;S.NTED;CASSE.
569,TRBV10-2*01,human,B,V,gatgctggaatcacccagagcccaagatacaagatcacagagacag...,1,DAGITQSPRYKITETGRQVTLMCHQTWSH.......SYMFWYRQDL...,27-38;56-65;81-86;104-109,WSH.......SY;SAA....ADI;S.KTEN;CASSE.
570,TRBV10-2*02,human,B,V,aaggcaggtgaccttgatgtgtcaccagacttggagccacagctat...,2,................RQVTLMCHQTWSH.......SYMFWYRQDL...,27-38;56-65;81-86;104-109,WSH.......SY;SAA....ADI;S.KTEN;......
571,TRBV10-3*01,human,B,V,gatgctggaatcacccagagcccaagacacaaggtcacagagacag...,1,DAGITQSPRHKVTETGTPVTLRCHQTENH.......RYMYWYRQDP...,27-38;56-65;81-86;104-109,ENH.......RY;SYG....VKD;S.KTED;CAISE.


In [11]:
combo_xcr_ids = list(set(V_sub_info.id))
combo_xcr_ids.sort()
combo_xcr_ids

['TRBV1*01',
 'TRBV10-1*01',
 'TRBV10-1*02',
 'TRBV10-2*01',
 'TRBV10-2*02',
 'TRBV10-3*01',
 'TRBV10-3*02',
 'TRBV10-3*03',
 'TRBV10-3*04',
 'TRBV11-1*01',
 'TRBV11-2*01',
 'TRBV11-2*02',
 'TRBV11-2*03',
 'TRBV11-3*01',
 'TRBV11-3*02',
 'TRBV11-3*03',
 'TRBV12-1*01',
 'TRBV12-2*01',
 'TRBV12-3*01',
 'TRBV12-4*01',
 'TRBV12-4*02',
 'TRBV12-5*01',
 'TRBV13*01',
 'TRBV13*02',
 'TRBV14*01',
 'TRBV14*02',
 'TRBV15*01',
 'TRBV15*02',
 'TRBV15*03',
 'TRBV16*01',
 'TRBV16*02',
 'TRBV16*03',
 'TRBV17*01',
 'TRBV18*01',
 'TRBV19*01',
 'TRBV19*02',
 'TRBV19*03',
 'TRBV2*01',
 'TRBV2*02',
 'TRBV2*03',
 'TRBV20-1*01',
 'TRBV20-1*02',
 'TRBV20-1*03',
 'TRBV20-1*04',
 'TRBV20-1*05',
 'TRBV20-1*06',
 'TRBV20-1*07',
 'TRBV20/OR9-2*01',
 'TRBV20/OR9-2*02',
 'TRBV20/OR9-2*03',
 'TRBV21-1*01',
 'TRBV21/OR9-2*01',
 'TRBV23-1*01',
 'TRBV23/OR9-2*01',
 'TRBV23/OR9-2*02',
 'TRBV24-1*01',
 'TRBV24/OR9-2*01',
 'TRBV25-1*01',
 'TRBV26*01',
 'TRBV26/OR9-2*01',
 'TRBV26/OR9-2*02',
 'TRBV27*01',
 'TRBV28*01',
 'TR

In [12]:
to_combo_dict = defaultdict(str)

to_combo_dict['TRBV10-1'] = 'TRBV10-1*01'
to_combo_dict['TRBV10-2'] = 'TRBV10-2*01'
to_combo_dict['TRBV10-3'] = 'TRBV10-3*01'

to_combo_dict['TRBV11-1'] = 'TRBV11-1*01'
to_combo_dict['TRBV11-2'] = 'TRBV11-2*01'
to_combo_dict['TRBV11-3'] = 'TRBV11-3*01'

to_combo_dict['TRBV12-3'] = 'TRBV12-3*01'
to_combo_dict['TRBV12-4'] = 'TRBV12-4*01'
to_combo_dict['TRBV12-5'] = 'TRBV12-5*01'

to_combo_dict['TRBV13'] = 'TRBV13*01'
to_combo_dict['TRBV14'] = 'TRBV14*01'
to_combo_dict['TRBV15'] = 'TRBV15*01'
to_combo_dict['TRBV16'] = 'TRBV16*01'
to_combo_dict['TRBV18'] = 'TRBV18*01'
to_combo_dict['TRBV19'] = 'TRBV19*01'

to_combo_dict['TRBV2'] = 'TRBV2*01'
to_combo_dict['TRBV20-1'] = 'TRBV20-1*01'
to_combo_dict['TRBV21-1'] = 'TRBV21-1*01'
to_combo_dict['TRBV24-1'] = 'TRBV24-1*01'
to_combo_dict['TRBV25-1'] = 'TRBV25-1*01'
to_combo_dict['TRBV27'] = 'TRBV27*01'
to_combo_dict['TRBV28'] = 'TRBV28*01'
to_combo_dict['TRBV29-1'] = 'TRBV29-1*01'

to_combo_dict['TRBV3-1'] = 'TRBV3-1*01'
to_combo_dict['TRBV30'] = 'TRBV30*01'

to_combo_dict['TRBV4-1'] = 'TRBV4-1*01'
to_combo_dict['TRBV4-2'] = 'TRBV4-2*01'
to_combo_dict['TRBV4-3'] = 'TRBV4-3*01'

to_combo_dict['TRBV5-1'] = 'TRBV5-1*01'
to_combo_dict['TRBV5-4'] = 'TRBV5-4*01'
to_combo_dict['TRBV5-5'] = 'TRBV5-5*01'
to_combo_dict['TRBV5-6'] = 'TRBV5-6*01'
to_combo_dict['TRBV5-7'] = 'TRBV5-7*01'
to_combo_dict['TRBV5-8'] = 'TRBV5-8*01'

to_combo_dict['TRBV6-1'] = 'TRBV6-1*01'
to_combo_dict['TRBV6-3'] = 'TRBV6-3*01'
to_combo_dict['TRBV6-4'] = 'TRBV6-4*01'
to_combo_dict['TRBV6-5'] = 'TRBV6-5*01'
to_combo_dict['TRBV6-6'] = 'TRBV6-6*01'
to_combo_dict['TRBV6-7'] = 'TRBV6-7*01'
to_combo_dict['TRBV6-8'] = 'TRBV6-8*01'
to_combo_dict['TRBV6-9'] = 'TRBV6-9*01'

to_combo_dict['TRBV7-2'] = 'TRBV7-2*01'
to_combo_dict['TRBV7-3'] = 'TRBV7-3*01'
to_combo_dict['TRBV7-4'] = 'TRBV7-4*01'
to_combo_dict['TRBV7-6'] = 'TRBV7-6*01'
to_combo_dict['TRBV7-7'] = 'TRBV7-7*01'
to_combo_dict['TRBV7-8'] = 'TRBV7-8*01'
to_combo_dict['TRBV7-9'] = 'TRBV7-9*01'

to_combo_dict['TRBV9'] = 'TRBV9*01'

In [13]:
len(to_combo_dict)

50

In [14]:
set(to_combo_dict.keys()) == set(all_vs)

True

In [15]:
# verify whether everything is simply by adding *01
flag_list = []

for v in all_vs:
    flag_list += [to_combo_dict[v] == v + "*01"]

sum(flag_list)/len(flag_list)

1.0

In [16]:
# verify whether all values belong to the ids in combo_xcr.tsv
set(to_combo_dict.values()) - set(V_sub_info.id)

set()

In [17]:
# look into the characters among amino acid sequences
Counter("".join(pos_aa + neg_aa))

Counter({'C': 27446,
         'S': 59485,
         'A': 40713,
         'H': 4792,
         'G': 39691,
         'F': 40384,
         'N': 11683,
         'E': 21312,
         'Q': 26082,
         'P': 11576,
         'L': 15686,
         'D': 10281,
         'Y': 22564,
         'R': 15644,
         'T': 25597,
         'K': 3461,
         'V': 6753,
         'M': 989,
         'I': 4144,
         'W': 2085})

In [18]:
len(set("".join(pos_aa + neg_aa)))

20

In [19]:
AA_SYMOLS = ['A', 'R', 'N', 'D', 'C',
             'Q', 'E', 'G', 'H', 'I',
             'L', 'K', 'M', 'F', 'P',
             'S', 'T', 'W', 'Y', 'V']

set("".join(pos_aa + neg_aa)) == set(AA_SYMOLS)

True

In [ ]:
# by using the cdr1, cdr2, cdr2.5 sequences for V part, we can allow more v genes as long
# as they are allowed for tcrdist

In [20]:
df_pos[:6]

,tcr
0,"TRBV20-1,CSAHGGFANHHEQFF"
1,"TRBV20-1,CSAHGGFANHHEQFF"
2,"TRBV20-1,CSAHGGFANHHEQFF"
3,"TRBV20-1,CSAHGGFANHHEQFF"
4,"TRBV20-1,CSAHGGFANHHEQFF"
5,"TRBV20-1,CSAHGGFANHHEQFF"


In [21]:
df_pos["v_forward"] = [to_combo_dict[x] for x in pos_v]
df_neg["v_forward"] = [to_combo_dict[x] for x in neg_v]

df_pos["aa"] = pos_aa
df_neg["aa"] = neg_aa

df_pos[:6]

,tcr,v_forward,aa
0,"TRBV20-1,CSAHGGFANHHEQFF",TRBV20-1*01,CSAHGGFANHHEQFF
1,"TRBV20-1,CSAHGGFANHHEQFF",TRBV20-1*01,CSAHGGFANHHEQFF
2,"TRBV20-1,CSAHGGFANHHEQFF",TRBV20-1*01,CSAHGGFANHHEQFF
3,"TRBV20-1,CSAHGGFANHHEQFF",TRBV20-1*01,CSAHGGFANHHEQFF
4,"TRBV20-1,CSAHGGFANHHEQFF",TRBV20-1*01,CSAHGGFANHHEQFF
5,"TRBV20-1,CSAHGGFANHHEQFF",TRBV20-1*01,CSAHGGFANHHEQFF


In [22]:
df_pos.shape

(6547, 3)

In [23]:
len(set([(v, aa) for v, aa in zip(df_pos.v_forward.tolist(), df_pos.aa.tolist())]))

4901

In [25]:
len(set(df_pos.v_forward))

49

In [24]:
Counter(df_pos.v_forward)

Counter({'TRBV20-1*01': 885,
         'TRBV27*01': 182,
         'TRBV6-5*01': 269,
         'TRBV7-7*01': 31,
         'TRBV2*01': 287,
         'TRBV5-6*01': 180,
         'TRBV5-1*01': 441,
         'TRBV28*01': 320,
         'TRBV9*01': 191,
         'TRBV5-4*01': 189,
         'TRBV18*01': 261,
         'TRBV19*01': 346,
         'TRBV14*01': 53,
         'TRBV24-1*01': 81,
         'TRBV6-4*01': 28,
         'TRBV7-2*01': 260,
         'TRBV10-1*01': 43,
         'TRBV7-6*01': 83,
         'TRBV3-1*01': 184,
         'TRBV11-2*01': 145,
         'TRBV12-3*01': 136,
         'TRBV30*01': 126,
         'TRBV4-1*01': 122,
         'TRBV7-9*01': 220,
         'TRBV11-3*01': 65,
         'TRBV29-1*01': 96,
         'TRBV4-3*01': 57,
         'TRBV6-1*01': 258,
         'TRBV7-8*01': 86,
         'TRBV4-2*01': 87,
         'TRBV5-8*01': 29,
         'TRBV10-3*01': 129,
         'TRBV15*01': 87,
         'TRBV5-5*01': 60,
         'TRBV6-3*01': 122,
         'TRBV6-6*01': 100,
         

In [26]:
df_neg.shape

(20860, 3)

In [27]:
len(set([(v, aa) for v, aa in zip(df_neg.v_forward.tolist(), df_neg.aa.tolist())]))

17042

In [28]:
len(set(df_neg.v_forward))

49

In [34]:
set(df_pos.v_forward) - set(df_neg.v_forward)

{'TRBV6-9*01'}

In [35]:
set(df_neg.v_forward) - set(df_pos.v_forward)

{'TRBV7-4*01'}

In [29]:
Counter(df_neg.v_forward)

Counter({'TRBV6-5*01': 789,
         'TRBV12-3*01': 594,
         'TRBV7-6*01': 191,
         'TRBV5-1*01': 1434,
         'TRBV7-2*01': 1114,
         'TRBV20-1*01': 2501,
         'TRBV12-4*01': 336,
         'TRBV7-9*01': 719,
         'TRBV4-2*01': 259,
         'TRBV28*01': 1133,
         'TRBV7-8*01': 339,
         'TRBV3-1*01': 685,
         'TRBV2*01': 1080,
         'TRBV15*01': 201,
         'TRBV18*01': 712,
         'TRBV11-2*01': 508,
         'TRBV10-3*01': 423,
         'TRBV19*01': 1106,
         'TRBV29-1*01': 628,
         'TRBV10-2*01': 166,
         'TRBV6-1*01': 717,
         'TRBV7-3*01': 303,
         'TRBV5-4*01': 393,
         'TRBV9*01': 598,
         'TRBV25-1*01': 122,
         'TRBV4-1*01': 320,
         'TRBV21-1*01': 48,
         'TRBV30*01': 536,
         'TRBV4-3*01': 169,
         'TRBV5-6*01': 333,
         'TRBV5-8*01': 90,
         'TRBV14*01': 205,
         'TRBV24-1*01': 245,
         'TRBV13*01': 54,
         'TRBV27*01': 473,
         'TRBV5-5*0

In [30]:
output_pos = pd.DataFrame(list(zip(df_pos.v_forward.tolist(), 
                                   df_pos.aa.tolist())), columns = ["v", "aa"])
output_neg = pd.DataFrame(list(zip(df_neg.v_forward.tolist(), 
                                   df_neg.aa.tolist())), columns = ["v", "aa"])

output_pos.to_csv(data_dir + "CD4_pos_TCRs_weighted_for_DePTH_comb_xcr_format.csv", index = False)
output_neg.to_csv(data_dir + "CD4_neg_TCRs_weighted_for_DePTH_comb_xcr_format.csv", index = False)

In [31]:
output_pos.shape

(6547, 2)

In [32]:
output_neg.shape

(20860, 2)